## 库

In [1]:
import base64
from time import sleep, time
import pymysql

In [2]:
from driverOps import getDriver, waitTillOpen, waitTillLoaded, closeAllOther
from AminerScholarExtracter import AminerScholarExtracter
from AminerSearchExtracter import searchName, getScholarPage, getScholarIds
from utils import stdAminerId, typeOfScript
from driverUtils import getTextByXpath, getTextsByXpath, getAttributeByXpath, getTextById
from cfg import tqdmNcols as ncols, mysqlIp, mysqlPort, mysqlUser, mysqlPassword, mysqlDb, waitUnit, waitLoading, scholarGap
from MysqlInserter import addCursor, checkScholar, checkDoneName, insertScholar, insertDoneName
from MysqlInserter import insertAwards, insertPortrait, insertTalentPools, insertWebpages, insertPapers, insertPublish
from namelist import names

In [3]:
from vUtil.vFile import fprint
from vUtil.vEmail import sendEmail
from vUtil.vTime import getNow, convertSeconds

In [4]:
from PIL import Image
import io
from vVariableInspector import _getshapeof

In [5]:
if typeOfScript() == 'jupyter':
    from tqdm.notebook import tqdm, trange
else:
    from tqdm import tqdm, trange
from time import sleep

In [6]:
def b64pic2img (pic):
    if pic is None: return None
    return Image.open(io.BytesIO(base64.b64decode(pic)))

In [7]:
def ourLog (name, log, logType = ''):
    fprint(f'[LOG] ({getNow()}) {repr(name)} : {logType} : "{log}"\n', file='log.txt', path='log')

## 爬取信息

### 打开浏览器

In [14]:
driver = getDriver(0)

### 连接数据库

In [15]:
db = pymysql.connect(mysqlIp, mysqlUser, mysqlPassword,port=mysqlPort,charset='utf8',db=mysqlDb)
cursor = db.cursor()
addCursor(cursor)

### 采集

In [10]:
names = [1]

In [16]:
now = 0
#names = [name for name in names if not checkDoneName(name)]
for name in tqdm(names, ncols=ncols):
#     waitTime = waitUnit
#     iWait = 0
#     while 1:
#         if not searchName(driver, name):
#             sleep(waitTime)
#             iWait += 1
#             if waitTime < 2 * waitLoading: waitTime += waitUnit
#             else: ourLog(name, iWait, 'list open failed')
#             continue
#         waitTime = waitUnit
#         ids = getScholarIds(driver, name)
#         if ids is not None: break
#         iWait += 1
    
#     ids = [id for id in ids if not checkScholar(id)]
    ids = ['5612639d45ce1e5962bff0ba']
    for id in tqdm(ids, ncols=ncols, leave=False):
        waitTime = waitUnit
        iWait = 0
        print (id)
        
        while time() < now + scholarGap: sleep(1)
        now = time()
        while 1:
            try:
                getScholarPage(driver, id)
                ex = AminerScholarExtracter(driver)

                name, pinyin = ex.getName()
                portrait = ex.getPortrait()
                title = ex.getTitle()
                department = ex.getDepartment()
                email = ex.getEmail()
                phone = ex.getPhone()
                fax = ex.getFax()
                address = ex.getAddress()
                webpages = ex.getWebPages()
                awards = ex.getAwards()
                talentPools = ex.getTalentPools()
                experience = ex.getExperience()
                education = ex.getEducation()
                brief = ex.getBrief()
                papers = ex.getPapers()
            except KeyboardInterrupt as e:
                raise e
            except Exception as e:
                sleep(waitTime)
                iWait += 1
                if waitTime < 2 * waitLoading: waitTime += waitUnit
                ourLog(name, f'id({id}) wait({iWait})', str(e))
                if str(e) == 'load failed' or str(e) == 'no sort by reference':
                    continue
                raise e
            insertScholar(id, name, pinyin, phone, title, department, fax, email, address, experience, education, brief)
            insertPortrait(id, portrait)
            insertWebpages(id, webpages)
            insertAwards(id, awards)
            insertTalentPools(id, talentPools)
            insertPapers(papers)
            insertPublish(id, papers)
            db.commit()
            closeAllOther(driver)
            break
    insertDoneName(name)
    db.commit()

5612639d45ce1e5962bff0ba



HTTPError: HTTP Error 404: Not Found

In [21]:
from MysqlInserter import select

In [23]:
rst = select('distinct display_name as name', 'landinn.authors')
len(rst)

87053

In [33]:
def solve (x, file):
    fprint('names = [\n', file)
    for y in tqdm(x, leave=False):
        fprint(f'{repr(y)},\n', file)
    fprint(']', file)

In [34]:
step = 10000
for now in trange(0, len(rst), step):
    solve(rst['name'][now:now+step], f'{now}.py')

### 关闭数据库

In [13]:
db.close()

### 关闭浏览器

In [12]:
driver.quit()